# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [107]:
# import libraries
import pandas as pd
import numpy as np
import nltk
import re
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_score,recall_score,precision_recall_fscore_support
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from nltk.corpus import stopwords
import pickle

nltk.download(['punkt', 'wordnet','stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\17711\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\17711\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\17711\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DRData.db')
df = pd.read_sql_table('ClassifiedMessages', engine)
X = df.message.values
Y = df[df.columns[4:]].values

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    #replacing urls
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    for i in urls:
        text = text.replace(i, "urlplaceholder")
        
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #Tokenize words
    words = word_tokenize(text)
    
    #Lemmatize words
    lemmed = [WordNetLemmatizer().lemmatize(w.strip()) for w in words]
    
    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [49]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ]))
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10,random_state = 0)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X, Y,random_state = 0)
model = pipeline
model.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma...            oob_score=False, random_state=0, verbose=0, warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [20]:
def show_results(Y_test,X_pred,columns):
    eval_dict={'column':[],'f1_score':[],'precision':[],'recall':[]}
    
    for column in range(len(columns)):
        f1s=f1_score(Y_test[:,column],X_pred[:,column], average='weighted')
        prs=precision_score(Y_test[:,column],X_pred[:,column], average='weighted')
        res=recall_score(Y_test[:,column],X_pred[:,column], average='weighted')
        eval_dict['column'].append(columns[column])
        eval_dict['f1_score'].append(f1s)
        eval_dict['precision'].append(prs)
        eval_dict['recall'].append(res)
        
    return pd.DataFrame.from_dict(eval_dict)

In [21]:
col_names=df.columns[4:]

x_train, x_test, y_train, y_test = train_test_split(X, Y,random_state = 0)

x_pred = model.predict(x_test)

results=show_results(y_test,x_pred,col_names)

results

,column,f1_score,precision,recall
0,related,0.776814,0.776873,0.797833
1,request,0.854850,0.871596,0.876106
2,offer,0.994129,0.992384,0.995880
3,aid_related,0.719979,0.736970,0.731309
4,medical_help,0.894251,0.901820,0.922490
5,medical_products,0.934434,0.940082,0.952243
6,search_and_rescue,0.957746,0.956493,0.970095
7,security,0.974818,0.983351,0.983064
8,military,0.957978,0.963229,0.969484
9,child_alone,1.000000,1.000000,1.000000


In [22]:
results.describe()

,f1_score,precision,recall
count,36.000000,36.000000,36.000000
mean,0.927601,0.932124,0.941677
std,0.066107,0.062158,0.060398
min,0.719979,0.736970,0.731309
25%,0.913720,0.920536,0.927983
50%,0.939633,0.949865,0.956286
75%,0.975794,0.974734,0.983445
max,1.000000,1.000000,1.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [41]:
def tune_model(ML_Model,parameters):
    Gcv = GridSearchCV(ML_Model, param_grid=parameters, cv=3, verbose=10, n_jobs=-1)
    
    return Gcv

In [50]:
parameters_RF = {
    'features__text_pipeline__vect__max_df': (0.75, 1.0),
    'features__text_pipeline__tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [50,75,100],
    'clf__estimator__max_depth': [None, 10,15]
}

cv=tune_model(pipeline,parameters_RF)
cv.fit(x_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 17.2min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 19.6min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed: 21.1min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 23.1min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed: 24.7min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed: 25.8min
[Parallel(n_jobs=-1)]: Done 104 out of 108 | elapsed: 28.6min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed: 28.8min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma...            oob_score=False, random_state=0, verbose=0, warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'features__text_pipeline__vect__max_df': (0.75, 1.0), 'features__text_pipeline__tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [50, 75, 100], 'clf__estimator__max_depth': [None, 10, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [51]:
cv.best_params_

{'clf__estimator__max_depth': None,
 'clf__estimator__n_estimators': 100,
 'features__text_pipeline__tfidf__use_idf': True,
 'features__text_pipeline__vect__max_df': 0.75}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [55]:
x_pred_Improved_Model = cv.predict(x_test)

results_Improved_Model=show_results(y_test,x_pred_Improved_Model,col_names)
results_Improved_Model

,column,f1_score,precision,recall
0,related,0.772468,0.791407,0.806225
1,request,0.871929,0.888721,0.889075
2,offer,0.994129,0.992384,0.995880
3,aid_related,0.759055,0.765918,0.764419
4,medical_help,0.885160,0.890960,0.920049
5,medical_products,0.932374,0.941499,0.951938
6,search_and_rescue,0.957286,0.963342,0.970705
7,security,0.974440,0.966114,0.982911
8,military,0.954769,0.964393,0.968569
9,child_alone,1.000000,1.000000,1.000000


Initial Model Results

In [53]:
results.describe()

,f1_score,precision,recall
count,36.000000,36.000000,36.000000
mean,0.927601,0.932124,0.941677
std,0.066107,0.062158,0.060398
min,0.719979,0.736970,0.731309
25%,0.913720,0.920536,0.927983
50%,0.939633,0.949865,0.956286
75%,0.975794,0.974734,0.983445
max,1.000000,1.000000,1.000000


Improved Model Results

In [54]:
results_Improved_Model.describe()

,f1_score,precision,recall
count,36.000000,36.000000,36.000000
mean,0.931308,0.936249,0.945500
std,0.060456,0.054905,0.053821
min,0.759055,0.765918,0.764419
25%,0.919329,0.919487,0.931073
50%,0.939311,0.952666,0.956286
75%,0.975287,0.974716,0.983255
max,1.000000,1.000000,1.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [77]:
pipeline_V2 = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ]))
    ])),
    ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC(random_state = 0))))
])

In [88]:
model_V2 = pipeline_V2
model_V2.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma...'l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=None),
           n_jobs=None))])

In [87]:
x_pred_V2 = model.predict(x_test)

results_V2=show_results(y_test,x_pred_V2,col_names)

results_V2.describe()

,f1_score,precision,recall
count,36.000000,36.000000,36.000000
mean,0.943168,0.942406,0.949492
std,0.054613,0.054969,0.052812
min,0.758245,0.758125,0.759536
25%,0.932660,0.930349,0.944347
50%,0.962899,0.962391,0.965670
75%,0.981885,0.980591,0.985505
max,1.000000,1.000000,1.000000


In [89]:
parameters_SVC = {
    'features__text_pipeline__vect__max_df': (0.5,0.75, 1.0),
    'features__text_pipeline__tfidf__use_idf': (True, False),
    'clf__estimator__estimator__loss': ['hinge','squared_hinge'],
    'clf__estimator__estimator__multi_class': ['ovr', 'crammer_singer'],
    'clf__estimator__estimator__max_iter': [1000,2000,5000]
}

cv_SVC=tune_model(pipeline_V2,parameters_SVC)
cv_SVC.fit(x_train, y_train)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   36.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed: 10.3min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 11.6min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed: 13.0min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 14

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma...'l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=None),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0), 'features__text_pipeline__tfidf__use_idf': (True, False), 'clf__estimator__estimator__loss': ['hinge', 'squared_hinge'], 'clf__estimator__estimator__multi_class': ['ovr', 'crammer_singer'], 'clf__estimator__estimator__max_iter': [1000, 2000, 5000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [90]:
cv_SVC.best_params_

{'clf__estimator__estimator__loss': 'hinge',
 'clf__estimator__estimator__max_iter': 1000,
 'clf__estimator__estimator__multi_class': 'crammer_singer',
 'features__text_pipeline__tfidf__use_idf': False,
 'features__text_pipeline__vect__max_df': 0.5}

In [91]:
x_pred_Improved_Model_SVC = cv_SVC.predict(x_test)

results_Improved_Model_SVC=show_results(y_test,x_pred_Improved_Model_SVC,col_names)

results_Improved_Model_SVC.describe()

,f1_score,precision,recall
count,36.000000,36.000000,36.000000
mean,0.943578,0.944653,0.951582
std,0.052757,0.050747,0.049044
min,0.779865,0.780789,0.781813
25%,0.929442,0.935205,0.945529
50%,0.962549,0.962561,0.965822
75%,0.980905,0.978059,0.984857
max,1.000000,1.000000,1.000000


In [93]:
def tokenize_without_stopW(text):
    #replacing urls
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    for i in urls:
        text = text.replace(i, "urlplaceholder")
        
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #Tokenize words
    words = word_tokenize(text)
    
    #Lemmatize words
    lemmed = [WordNetLemmatizer().lemmatize(w.strip()) for w in words]
    
    # remove stopwords
    STOPWORDS = list(set(stopwords.words('english')))
    clean_tokens = [token for token in lemmed if token not in STOPWORDS]
    
    return clean_tokens

In [100]:
pipeline_V3 = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize_without_stopW)),
            ('tfidf', TfidfTransformer())
        ]))
    ])),
    ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC(random_state = 0))))
])

In [103]:
model_V3 = pipeline_V3
model_V3.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma...'l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=None),
           n_jobs=None))])

In [104]:
x_pred_V3 = model.predict(x_test)

results_V3=show_results(y_test,x_pred_V3,col_names)

results_V3.describe()

,f1_score,precision,recall
count,36.000000,36.000000,36.000000
mean,0.943168,0.942406,0.949492
std,0.054613,0.054969,0.052812
min,0.758245,0.758125,0.759536
25%,0.932660,0.930349,0.944347
50%,0.962899,0.962391,0.965670
75%,0.981885,0.980591,0.985505
max,1.000000,1.000000,1.000000


### 9. Export your model as a pickle file

In [108]:
pickle.dump(model_V2, open('classifier.pkl', "wb"))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.